In [3]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [7]:
db_loc = r"C:\Users\Jonathan Gupton\DataspellProjects\nyt-crossword-analysis\01-build-the-dataset\crossword.db"
conn = sqlite3.connect(db_loc)
df = pd.read_sql("select * from crossword_url", con=conn)
conn.close()

In [15]:
def fetch_datasource(query: str) -> pd.DataFrame:
    db_loc = r"C:\Users\Jonathan Gupton\DataspellProjects\nyt-crossword-analysis\01-build-the-dataset\crossword.db"
    conn = sqlite3.connect(db_loc)
    df = pd.read_sql(query, con=conn)
    conn.close()
    return df

query = """
SELECT cw.date,
       length(a.answer)        AS answer_length
FROM answers_date ad
         INNER JOIN answers a ON ad.answer_key = a.pk
         INNER JOIN crossword_url cw ON cw.pk = ad.date_key
"""
df = fetch_datasource(query)
df['date'] = pd.to_datetime(df['date'], format="%m-%d-%y")
df['day_of_week'] = df['date'].dt.day_name()

In [16]:
df.head()

,date,answer_length,day_of_week
0,2010-05-30,4,Sunday
1,2010-05-30,5,Sunday
2,2010-05-30,5,Sunday
3,2010-05-30,3,Sunday
4,2010-05-30,5,Sunday


In [55]:
cts = df.assign(n=1).groupby(['date', 'day_of_week', 'answer_length']).count().unstack(level=-1).fillna(0)
cts.columns = cts.columns.droplevel(0)
cts

,answer_length,0,1,2,3,4,5,6,7,8,9,...,17,18,19,20,21,22,23,29,31,42
date,day_of_week,,,,,,,,,,,,,,,,,,,,,
2010-05-30,Sunday,0.0,0.0,0.0,49.0,61.0,46.0,37.0,36.0,12.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2010-11-19,Friday,0.0,0.0,0.0,2.0,24.0,12.0,12.0,10.0,6.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-01-07,Friday,0.0,0.0,0.0,14.0,14.0,8.0,10.0,16.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-03-17,Thursday,0.0,0.0,0.0,23.0,17.0,20.0,9.0,0.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2011-04-10,Sunday,0.0,0.0,0.0,19.0,38.0,40.0,14.0,6.0,8.0,4.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-03,Sunday,0.0,0.0,0.0,36.0,28.0,30.0,22.0,11.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-11-04,Monday,0.0,0.0,0.0,21.0,24.0,13.0,8.0,2.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2024-11-05,Tuesday,0.0,0.0,0.0,19.0,32.0,14.0,6.0,0.0,2.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
pcts = cts.div(cts.sum(axis=1), axis=0)
pcts = pcts.reset_index()
pcts

answer_length,date,day_of_week,0,1,2,3,4,5,6,7,...,17,18,19,20,21,22,23,29,31,42
0,2010-05-30,Sunday,0.0,0.0,0.0,0.190661,0.237354,0.178988,0.143969,0.140078,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,2010-11-19,Friday,0.0,0.0,0.0,0.029412,0.352941,0.176471,0.176471,0.147059,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,2011-01-07,Friday,0.0,0.0,0.0,0.200000,0.200000,0.114286,0.142857,0.228571,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,2011-03-17,Thursday,0.0,0.0,0.0,0.287500,0.212500,0.250000,0.112500,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,2011-04-10,Sunday,0.0,0.0,0.0,0.139706,0.279412,0.294118,0.102941,0.044118,...,0.007353,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2549,2024-11-03,Sunday,0.0,0.0,0.0,0.257143,0.200000,0.214286,0.157143,0.078571,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2550,2024-11-04,Monday,0.0,0.0,0.0,0.276316,0.315789,0.171053,0.105263,0.026316,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2551,2024-11-05,Tuesday,0.0,0.0,0.0,0.243590,0.410256,0.179487,0.076923,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2552,2024-11-06,Wednesday,0.0,0.0,0.0,0.246914,0.333333,0.172840,0.135802,0.024691,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [68]:
(pcts.groupby(['day_of_week',pcts.date.dt.year]).mean()[3] + pcts.groupby([pcts.date.dt.year, 'day_of_week']).mean()[4]).unstack().T 

day_of_week,Friday,Monday,Saturday,Sunday,Thursday,Tuesday,Wednesday
date,,,,,,,
2010,0.382353,NaN,NaN,0.428016,NaN,NaN,NaN
2011,0.400000,NaN,NaN,0.419118,0.500000,NaN,NaN
2012,NaN,0.538462,NaN,NaN,NaN,NaN,NaN
2013,NaN,NaN,NaN,0.433824,NaN,NaN,NaN
2014,0.342857,0.512821,NaN,0.398661,0.421053,0.554054,NaN
2015,NaN,NaN,NaN,0.500000,0.506410,0.602564,0.500000
2016,NaN,NaN,NaN,NaN,0.512821,0.552632,0.512821
2017,0.442650,0.522942,0.322303,0.441968,0.524488,0.532764,0.489454
2018,0.401416,0.538162,0.361528,0.445784,0.495470,0.522698,0.518516


In [72]:
df.date.dt.year.unique()


array([2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020,
       2021, 2022, 2023, 2024], dtype=int32)

In [100]:
years = df.date.unique().year

In [101]:
from collections import Counter

c = Counter(years)

In [102]:
c

Counter({np.int32(2020): 366,
         np.int32(2021): 365,
         np.int32(2022): 365,
         np.int32(2023): 365,
         np.int32(2019): 360,
         np.int32(2018): 355,
         np.int32(2024): 312,
         np.int32(2017): 45,
         np.int32(2014): 6,
         np.int32(2015): 5,
         np.int32(2011): 3,
         np.int32(2016): 3,
         np.int32(2010): 2,
         np.int32(2012): 1,
         np.int32(2013): 1})